# Task 13.2: Custom Tools - Solutions

This notebook contains complete solutions for the exercises in the Custom Tools notebook.

---

## Exercise: Create Your Own Custom Tool

**Solution:** Here are three custom tools: Date/Time, Unit Converter, and Random Data Generator.

In [ ]:
from langchain.tools import tool
from datetime import datetime, timedelta
import random
import json

# Tool 1: Date/Time Tool
@tool
def get_datetime(format: str = "%Y-%m-%d %H:%M:%S") -> str:
    """
    Get the current date and time.
    
    Use this when you need to know the current date or time.
    
    Args:
        format: strftime format string. Common formats:
            - "%Y-%m-%d" for date (2024-01-15)
            - "%H:%M:%S" for time (14:30:00)
            - "%A, %B %d, %Y" for full date (Monday, January 15, 2024)
    
    Returns:
        Current datetime as formatted string
    """
    try:
        return datetime.now().strftime(format)
    except Exception as e:
        return f"Error: Invalid format string - {e}"

# Test
print("Date/Time Tool:")
print(f"  Default: {get_datetime.invoke('%Y-%m-%d %H:%M:%S')}")
print(f"  Date only: {get_datetime.invoke('%Y-%m-%d')}")
print(f"  Full: {get_datetime.invoke('%A, %B %d, %Y')}")

In [ ]:
# Tool 2: Unit Converter
@tool
def convert_units(value: str, from_unit: str, to_unit: str) -> str:
    """
    Convert between different units of measurement.
    
    Supports:
    - Length: km, m, cm, mm, mi, yd, ft, in
    - Weight: kg, g, lb, oz
    - Temperature: celsius, fahrenheit, kelvin
    - Data: TB, GB, MB, KB, B
    
    Args:
        value: The numeric value to convert
        from_unit: The source unit
        to_unit: The target unit
    
    Returns:
        Converted value with units
    """
    # Conversion factors to base units
    conversions = {
        # Length (base: meters)
        'km': 1000, 'm': 1, 'cm': 0.01, 'mm': 0.001,
        'mi': 1609.34, 'yd': 0.9144, 'ft': 0.3048, 'in': 0.0254,
        # Weight (base: grams)
        'kg': 1000, 'g': 1, 'lb': 453.592, 'oz': 28.3495,
        # Data (base: bytes)
        'tb': 1e12, 'gb': 1e9, 'mb': 1e6, 'kb': 1e3, 'b': 1,
    }
    
    try:
        num_value = float(value)
        from_lower = from_unit.lower()
        to_lower = to_unit.lower()
        
        # Handle temperature separately
        if from_lower in ['celsius', 'fahrenheit', 'kelvin']:
            if from_lower == 'celsius':
                base = num_value
            elif from_lower == 'fahrenheit':
                base = (num_value - 32) * 5/9
            else:  # kelvin
                base = num_value - 273.15
            
            if to_lower == 'celsius':
                result = base
            elif to_lower == 'fahrenheit':
                result = base * 9/5 + 32
            else:  # kelvin
                result = base + 273.15
            
            return f"{num_value} {from_unit} = {result:.2f} {to_unit}"
        
        # Regular conversion
        if from_lower not in conversions or to_lower not in conversions:
            return f"Error: Unknown unit '{from_unit}' or '{to_unit}'"
        
        # Convert to base then to target
        base_value = num_value * conversions[from_lower]
        result = base_value / conversions[to_lower]
        
        return f"{num_value} {from_unit} = {result:.4g} {to_unit}"
        
    except ValueError:
        return f"Error: '{value}' is not a valid number"

# Test
print("\nUnit Converter Tool:")
print(f"  {convert_units.invoke({'value': '100', 'from_unit': 'km', 'to_unit': 'mi'})}")
print(f"  {convert_units.invoke({'value': '32', 'from_unit': 'fahrenheit', 'to_unit': 'celsius'})}")
print(f"  {convert_units.invoke({'value': '128', 'from_unit': 'GB', 'to_unit': 'TB'})}")

In [ ]:
# Tool 3: Random Data Generator
@tool
def generate_random_data(data_type: str, count: str = "5") -> str:
    """
    Generate random sample data for testing and development.
    
    Args:
        data_type: Type of data - "names", "emails", "numbers", "uuids", "dates"
        count: Number of items to generate (default: 5, max: 20)
    
    Returns:
        JSON array of generated data
    """
    import uuid
    
    try:
        n = min(int(count), 20)  # Max 20 items
    except ValueError:
        n = 5
    
    first_names = ["Alice", "Bob", "Charlie", "Diana", "Eve", "Frank", "Grace", "Henry"]
    last_names = ["Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia", "Miller"]
    domains = ["example.com", "test.org", "demo.net"]
    
    data_type_lower = data_type.lower()
    
    if data_type_lower == "names":
        result = [f"{random.choice(first_names)} {random.choice(last_names)}" for _ in range(n)]
    elif data_type_lower == "emails":
        result = [f"{random.choice(first_names).lower()}.{random.choice(last_names).lower()}@{random.choice(domains)}" for _ in range(n)]
    elif data_type_lower == "numbers":
        result = [random.randint(1, 1000) for _ in range(n)]
    elif data_type_lower == "uuids":
        result = [str(uuid.uuid4()) for _ in range(n)]
    elif data_type_lower == "dates":
        base = datetime.now()
        result = [(base - timedelta(days=random.randint(0, 365))).strftime("%Y-%m-%d") for _ in range(n)]
    else:
        return f"Error: Unknown data type '{data_type}'. Use: names, emails, numbers, uuids, dates"
    
    return json.dumps(result, indent=2)

# Test
print("\nRandom Data Generator Tool:")
print(f"Names: {generate_random_data.invoke({'data_type': 'names', 'count': '3'})}")
print(f"\nEmails: {generate_random_data.invoke({'data_type': 'emails', 'count': '3'})}")
print(f"\nUUIDs: {generate_random_data.invoke({'data_type': 'uuids', 'count': '2'})}")

## Challenge: Multi-Step Agent Task

**Solution:** Agent that searches, writes code, and calculates.

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama

# Import tools from previous cells (assume they're defined)
# Plus our existing tools from the main notebook

@tool
def calculate(expression: str) -> str:
    """Calculate a mathematical expression. Example: calculate('128 / 14')."""
    try:
        return str(eval(expression))  # Simplified for demo
    except:
        return "Error calculating"

@tool
def search_knowledge(query: str) -> str:
    """Search for information about DGX Spark."""
    kb = {
        "memory": "DGX Spark has 128GB unified memory",
        "model size": "A 7B parameter model in FP16 uses about 14GB (7B * 2 bytes)",
    }
    for key, val in kb.items():
        if key in query.lower():
            return val
    return "No relevant information found."

# Create agent
llm = Ollama(model="llama3.1:8b", temperature=0.1)
tools = [calculate, search_knowledge]

prompt = PromptTemplate.from_template(
"""Answer the question using the available tools.

Tools: {tools}

Format:
Question: the question
Thought: think about what to do
Action: tool name
Action Input: input to tool
Observation: result
... (repeat as needed)
Thought: I know the answer
Final Answer: the answer

Question: {input}
Thought:{agent_scratchpad}""")

agent = create_react_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=5)

# Run the complex query
result = executor.invoke({
    "input": "How many 7B parameter models can fit in DGX Spark's memory?"
})

print(f"\nFinal Answer: {result['output']}")

## Key Takeaways

1. **Good tool design:**
   - Clear, specific descriptions
   - Type hints for arguments
   - Error handling
   - Examples in docstrings

2. **Security considerations:**
   - Validate inputs
   - Limit capabilities
   - Don't use `eval()` in production

3. **Agent integration:**
   - Tools should have single responsibility
   - Return informative error messages
   - Consider the agent's perspective